In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

from sqlalchemy import create_engine

# AWS Server password in config.py file
from config import db_password

# Direction to AWS RDS Server
url_AWS = "dbproject.cprghtxfxmsc.us-east-1.rds.amazonaws.com"

# Database name in Server
db_name = "postgres"

# Connection string
db_string = f"postgresql://postgres:{db_password}@{url_AWS}:5432/{db_name}"

db = create_engine(db_string)

In [2]:
# Read
Credit_result = db.execute("SELECT * FROM credit_join")
print(Credit_result)

In [3]:
application_df = pd.DataFrame(Credit_result,columns=('ID','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','CNT_CHILDREN','AMT_INCOME_TOTAL','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','DAYS_BIRTH','DAYS_EMPLOYED','FLAG_MOBIL','FLAG_WORK_PHONE','FLAG_PHONE','FLAG_EMAIL','JOB','BEGIN_MONTHS','STATUS','TARGET'))
application_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,JOB,BEGIN_MONTHS,STATUS,TARGET
0,5091379,F,Y,Y,2+ children,"$216,000.00",Higher education,Married,House / apartment,-12757,-4887,True,False,False,False,Core staff,-20,0,0
1,5146079,F,N,Y,1 children,"$108,000.00",Secondary / secondary special,Married,House / apartment,-12723,-1132,True,True,True,False,Sales staff,-44,0,0
2,5023748,M,Y,N,No children,"$157,500.00",Secondary / secondary special,Married,House / apartment,-11454,-1604,True,False,False,False,Drivers,-10,C,0
3,5038545,M,Y,N,No children,"$225,000.00",Secondary / secondary special,Married,House / apartment,-9479,-1143,True,False,False,False,Laborers,-8,C,0
4,5022720,M,Y,Y,2+ children,"$157,500.00",Secondary / secondary special,Married,House / apartment,-15202,-674,True,False,False,False,Drivers,-10,0,0


In [4]:
application_df['ID'].count()

537667

In [5]:
application_df.dtypes

ID                      int64
CODE_GENDER            object
FLAG_OWN_CAR           object
FLAG_OWN_REALTY        object
CNT_CHILDREN           object
AMT_INCOME_TOTAL       object
NAME_EDUCATION_TYPE    object
NAME_FAMILY_STATUS     object
NAME_HOUSING_TYPE      object
DAYS_BIRTH              int64
DAYS_EMPLOYED           int64
FLAG_MOBIL               bool
FLAG_WORK_PHONE          bool
FLAG_PHONE               bool
FLAG_EMAIL               bool
JOB                    object
BEGIN_MONTHS            int64
STATUS                 object
TARGET                  int64
dtype: object

In [6]:
# Find null values
for column in application_df.columns:
    print(f'Columns {column} has {application_df[column].isnull().sum()} nullvalues')

Columns ID has 0 nullvalues
Columns CODE_GENDER has 0 nullvalues
Columns FLAG_OWN_CAR has 0 nullvalues
Columns FLAG_OWN_REALTY has 0 nullvalues
Columns CNT_CHILDREN has 0 nullvalues
Columns AMT_INCOME_TOTAL has 0 nullvalues
Columns NAME_EDUCATION_TYPE has 0 nullvalues
Columns NAME_FAMILY_STATUS has 0 nullvalues
Columns NAME_HOUSING_TYPE has 0 nullvalues
Columns DAYS_BIRTH has 0 nullvalues
Columns DAYS_EMPLOYED has 0 nullvalues
Columns FLAG_MOBIL has 0 nullvalues
Columns FLAG_WORK_PHONE has 0 nullvalues
Columns FLAG_PHONE has 0 nullvalues
Columns FLAG_EMAIL has 0 nullvalues
Columns JOB has 0 nullvalues
Columns BEGIN_MONTHS has 0 nullvalues
Columns STATUS has 0 nullvalues
Columns TARGET has 0 nullvalues


In [7]:
# Determine the number of unique values in each column.
application_df.nunique()

ID                     25134
CODE_GENDER                2
FLAG_OWN_CAR               2
FLAG_OWN_REALTY            2
CNT_CHILDREN               3
AMT_INCOME_TOTAL         195
NAME_EDUCATION_TYPE        5
NAME_FAMILY_STATUS         5
NAME_HOUSING_TYPE          6
DAYS_BIRTH              5206
DAYS_EMPLOYED           3299
FLAG_MOBIL                 1
FLAG_WORK_PHONE            2
FLAG_PHONE                 2
FLAG_EMAIL                 2
JOB                       18
BEGIN_MONTHS              61
STATUS                     8
TARGET                     2
dtype: int64

In [8]:
# Drop the non-beneficial columns, 'DAYS_BIRTH', 'BEGIN_MONTHS'.
application_df.drop(['DAYS_BIRTH','BEGIN_MONTHS'],1, inplace =True)
application_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,JOB,STATUS,TARGET
0,5091379,F,Y,Y,2+ children,"$216,000.00",Higher education,Married,House / apartment,-4887,True,False,False,False,Core staff,0,0
1,5146079,F,N,Y,1 children,"$108,000.00",Secondary / secondary special,Married,House / apartment,-1132,True,True,True,False,Sales staff,0,0
2,5023748,M,Y,N,No children,"$157,500.00",Secondary / secondary special,Married,House / apartment,-1604,True,False,False,False,Drivers,C,0
3,5038545,M,Y,N,No children,"$225,000.00",Secondary / secondary special,Married,House / apartment,-1143,True,False,False,False,Laborers,C,0
4,5022720,M,Y,Y,2+ children,"$157,500.00",Secondary / secondary special,Married,House / apartment,-674,True,False,False,False,Drivers,0,0


In [9]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,CNT_CHILDREN_1 children,CNT_CHILDREN_2+ children,CNT_CHILDREN_No children,"AMT_INCOME_TOTAL_$1,125,000.00",...,JOB_Security staff,JOB_Waiters/barmen staff,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,ID,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,TARGET,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,...,JOB_Security staff,JOB_Waiters/barmen staff,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,5091379,-4887,True,False,False,False,0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5146079,-1132,True,True,True,False,0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5023748,-1604,True,False,False,False,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,5038545,-1143,True,False,False,False,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5022720,-674,True,False,False,False,0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df["TARGET"].values
X = application_df.drop(["TARGET"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                2530      
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 2,591
Trainable params: 2,591
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=10,callbacks=[cp_callback])

Epoch 1/10
12602/12602 [==============================] - 62s 983us/step - loss: 0.0032 - accuracy: 0.9993
Epoch 2/10
12602/12602 [==============================] - 12s 958us/step - loss: 4.3847e-05 - accuracy: 1.0000s - loss: 4
Epoch 3/10
12602/12602 [==============================] - 12s 970us/step - loss: 8.5332e-07 - accuracy: 1.0000
Epoch 4/10
12602/12602 [==============================] - 12s 974us/step - loss: 1.0947e-08 - accuracy: 1.0000
Epoch 5/10
12595/12602 [============================>.] - ETA: 0s - loss: 5.4934e-10 - accuracy: 1.0000
Epoch 00005: saving model to checkpoints\weights.05.hdf5
12602/12602 [==============================] - 16s 1ms/step - loss: 5.4906e-10 - accuracy: 1.0000
Epoch 6/10
12602/12602 [==============================] - 13s 1ms/step - loss: 5.4309e-11 - accuracy: 1.0000
Epoch 7/10
12602/12602 [==============================] - 13s 1ms/step - loss: 1.0047e-11 - accuracy: 1.0000
Epoch 8/10
12602/12602 [==============================] - 13s 1ms/step -

In [18]:
# Evaluate the model ux|sing the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4201/4201 - 5s - loss: 1.7334e-05 - accuracy: 1.0000 - 5s/epoch - 1ms/step
Loss: 1.733431599859614e-05, Accuracy: 0.9999850988388062
